In [ ]:
pip install geopandas

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import folium
from google.cloud import bigquery
from shapely import wkt
from google.colab import auth
auth.authenticate_user()
client = bigquery.Client(project='tarletondatascience2022')

query = """
SELECT
  z.zip_code,
  z.zip_code_geom AS polygon,
  COALESCE(e.hispanic_count, 0) AS hispanic_count, --hispanic count if null change to 0
  COALESCE(e.enrolled_in_grade_12, 0) AS enrolled_in_grade_12, --does the same thinf
  z.city,
  z.county
  /*
  CASE
    WHEN COALESCE(e.hispanic_count, 0) = 0 THEN 0 --0 or null then just lwave as zero
    ELSE COALESCE(e.enrolled_in_grade_12, 0) / COALESCE(e.hispanic_count, 1)  --otherwise just do calulation
  END AS ratio,
  CASE
    WHEN COALESCE(e.enrolled_in_grade_12, 0) = 0 THEN 0
    ELSE COALESCE(e.hispanic_count, 0) / COALESCE(e.enrolled_in_grade_12, 1) * 100
  END AS flipped_ratio --this is the ratio flipped total enrolled 12th/hispanic count */
FROM
  `bigquery-public-data.geo_us_boundaries.zip_codes` z
LEFT JOIN
  (SELECT
     hs.hs_zip,
     hs.hispanic_count,
     e.enrolled_in_grade_12
   FROM
     (SELECT
        CAST(hs_zip AS STRING) AS hs_zip,
        COUNT(*) AS hispanic_count
      FROM
        (SELECT
           hs_zip,
           hispanic,
           term_desc,
           styp_desc,
           CAST(SUBSTR(term_desc, -4) AS INT) - 0.5 * CASE WHEN term_desc LIKE '%Spring%' THEN 1 ELSE 0 END AS term
            -- find the year
           --   change back to integer
           --   - now subtract .5
         FROM
           `tarletondatascience2022.argil.tarleton data`
         WHERE
           term_desc != 'OnRamps 22-23') filtered_test2
      WHERE
        hispanic = 1
        AND styp_desc = 'New First Time'
        AND term BETWEEN 2018.5 AND 2021.5
      GROUP BY
        hs_zip) hs
   LEFT JOIN
     (SELECT
        CAST(zcta AS STRING) AS zcta,
        enrolled_in_grade_12
      FROM
        `tarletondatascience2022.argil.hispanic_enrolled_12`) e
   ON
     hs.hs_zip = e.zcta) e
ON
  z.zip_code = e.hs_zip
WHERE
  z.state_code = 'TX';
"""


df = client.query(query).to_dataframe()
df['hispanic_count'] = df['hispanic_count'] + 1
df['ratio'] = df['enrolled_in_grade_12'] / df['hispanic_count']
#df.to_csv('Qry.csv', index=False)

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import folium
from google.cloud import bigquery
from shapely import wkt
from folium.plugins import Search

# Load your data
# df = pd.read_csv('Qry.csv')

df['geometry'] = df['polygon'].apply(wkt.loads)

gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf.set_crs(epsg=4326, inplace=True)

m = folium.Map(location=[31.0, -100.0], zoom_start=6, tiles='CartoDB Positron')

thresholds = [0, 1, 2, 600, df['ratio'].max()]

folium.Choropleth(
    geo_data=gdf,
    name='choropleth',
    data=gdf,
    columns=['zip_code', 'ratio'],
    key_on='feature.properties.zip_code',
    fill_color='Purples',
    fill_opacity=0.4,
    line_opacity=0.1,
    threshold_scale=thresholds,
    legend_name='Ratio of Enrolled to Hispanic'
).add_to(m)

geojson = folium.GeoJson(
    gdf,
    style_function=lambda x: {
        'fillColor': '#ffffff',
        'color': '#000000',
        'weight': 0.3,
        'fillOpacity': 0.1,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['zip_code', 'city', 'county', 'hispanic_count', 'enrolled_in_grade_12', 'ratio'],
        aliases=['ZIP Code :', 'City :', 'County :', 'Hispanic Count :', 'Enrolled in Grade 12 :', '1 Hispanic student registers at Tarleton out of every :'],
        localize=True
    )
).add_to(m)

for attribute in ['zip_code', 'city', 'county']:
    Search(
        layer=geojson,
        geom_type='Polygon',
        placeholder=f'Search for a {attribute}',
        search_label=attribute,
        collapsed=False
    ).add_to(m)

folium.LayerControl().add_to(m)

m.save('texas_map.html')

m


In [ ]:
pip install census


In [ ]:
from census import Census
import pandas as pd

api_key = 'a026288c00e634cb64c4485ad778215df8979fb0'

c = Census(api_key)

variables = [
    'B14007I_013E']

data = c.acs5.state_county_tract(fields=variables, state_fips='48', county_fips='*', tract='*', year=2022)

df = pd.DataFrame(data)

df.rename(columns={
    'state': 'State',
    'county': 'County',
    'tract': 'Tract',
    'B14007I_013E': 'Hispanic_Enrolled_12th_Grade'
}, inplace=True)




df.to_csv('tract_hispanic_12th_grade_enrollment_2022.csv', index=False)


print(df.head())


   Hispanic_Enrolled_12th_Grade State County   Tract County_Name
0                          19.0    48    001  950100    Anderson
1                           0.0    48    001  950401    Anderson
2                           0.0    48    001  950402    Anderson
3                          42.0    48    001  950500    Anderson
4                          25.0    48    001  950600    Anderson


In [ ]:
import pandas as pd

df_census = pd.read_csv('/content/tract_hispanic_12th_grade_enrollment_with_county.csv')



df_texas_map = pd.read_csv( '/content/texas_map_prt3.csv')

df_texas_map['Census_Tract'] = df_texas_map['Census_Tract'].astype(str).str.zfill(6)
df_census['Tract'] = df_census['Tract'].astype(str).str.zfill(6)

df_census_selected = df_census[['Tract', 'Hispanic_Enrolled_12th_Grade', 'County']]

merged_df = df_texas_map.merge(df_census_selected, left_on='Census_Tract', right_on='Tract', how='left')

unmatched_df = merged_df[merged_df['Hispanic_Enrolled_12th_Grade'].isna()]


merged_df.to_csv('merged_texas_map_with_hispanic.csv', index=False)


unmatched_df.to_csv('unmatched_tracts.csv', index=False)

print("Merged DataFrame:")
print(merged_df.head())

print("\nUnmatched DataFrame:")
print(unmatched_df.head())


Merged DataFrame:
   TERM_DESC RANDOM_ID  HISPANIC                         MAJOR COLLEGE  \
0  Fall 2019  S1723716         1   Ag Services and Development      AN   
1  Fall 2021  S2098688         1  Wildlife Sus & Ecosystem Sci      AN   
2  Fall 2019  S1856328         1                Animal Science      AN   
3  Fall 2019  S1993678         1  Wildlife Sus & Ecosystem Sci      AN   
4  Fall 2019  S2006384         1                Animal Science      AN   

   TERM_HRS_ATMPT  TERM_HRS_ERND  TERM_GPA  OVRL_SCH  OVRL_GPA  ...  \
0              15             15      2.87        15  2.866667  ...   
1              15             15      2.53        15  2.533333  ...   
2              12              7      1.27        13  2.058824  ...   
3              15             11      4.00        17  4.000000  ...   
4              15              7      0.93         7  0.933333  ...   

          Street         City  State    Zip Longitude   Latitude  \
0  5005 STAHL RD  SAN ANTONIO     TX  7824

In [ ]:
import pandas as pd


df_grouped = merged_df.groupby('Census_Tract').agg({
    'HISPANIC': 'sum'
}).reset_index()

df_grouped.rename(columns={'HISPANIC': 'Total_HISPANIC'}, inplace=True)

final_df = merged_df.merge(df_grouped, on='Census_Tract', how='left')

final_df['ratio'] = final_df['Hispanic_Enrolled_12th_Grade'] / final_df['Total_HISPANIC']


final_df.to_csv('final_hispanic_enrollment_ratio_by_tract.csv', index=False)

print(final_df.head())


   TERM_DESC RANDOM_ID  HISPANIC                         MAJOR COLLEGE  \
0  Fall 2019  S1723716         1   Ag Services and Development      AN   
1  Fall 2021  S2098688         1  Wildlife Sus & Ecosystem Sci      AN   
2  Fall 2019  S1856328         1                Animal Science      AN   
3  Fall 2019  S1993678         1  Wildlife Sus & Ecosystem Sci      AN   
4  Fall 2019  S2006384         1                Animal Science      AN   

   TERM_HRS_ATMPT  TERM_HRS_ERND  TERM_GPA  OVRL_SCH  OVRL_GPA  ...  State  \
0              15             15      2.87        15  2.866667  ...     TX   
1              15             15      2.53        15  2.533333  ...     TX   
2              12              7      1.27        13  2.058824  ...     TX   
3              15             11      4.00        17  4.000000  ...     TX   
4              15              7      0.93         7  0.933333  ...     TX   

     Zip  Longitude   Latitude Census_Tract   Tract  \
0  78247  -98.39032  29.580601 

In [ ]:
import pandas as pd
import folium
from math import ceil


final_df['ratio'] = final_df['ratio'].apply(lambda x: ceil(x + 1))

m = folium.Map(location=[31.9686, -99.9018], zoom_start=6)

for index, row in final_df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=(
            f"City: {row['City']}<br>"
            f"Census Tract: {row['Census_Tract']}<br>"
            f"Hispanic Enrolled 12th Grade: {row['Hispanic_Enrolled_12th_Grade']}<br>"
            f"Total Hispanic: {row['Total_HISPANIC']}<br>"
            f"Ratio: {row['ratio']}"
        ),
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)


m.save('texas_census_tract_hispanic_map2.html')



'texas_census_tract_hispanic_map2.html'

In [ ]:
import pandas as pd
import folium
from math import ceil


final_df['ratio'] = final_df['ratio'].apply(lambda x: ceil(x + 1))

m = folium.Map(location=[31.9686, -99.9018], zoom_start=6)

for index, row in final_df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=(
            f"City: {row['City']}<br>"
            f"Census Tract: {row['Census_Tract']}<br>"
            f"Hispanic Enrolled 12th Grade: {row['Hispanic_Enrolled_12th_Grade']}<br>"
            f"Total Hispanic: {row['Total_HISPANIC']}<br>"
            f"Ratio: {row['ratio']}"
        ),
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)

m.save('texas_census_tract_hispanic_map3.html')



'texas_census_tract_hispanic_map3.html'

In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from math import ceil


final_df['ratio'] = final_df['ratio'].apply(lambda x: ceil(x + 1))

m = folium.Map(location=[31.9686, -99.9018], zoom_start=6)

marker_cluster = MarkerCluster().add_to(m)

for index, row in final_df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=(
            f"City: {row['City']}<br>"
            f"Census Tract: {row['Census_Tract']}<br>"
            f"Hispanic Enrolled 12th Grade: {row['Hispanic_Enrolled_12th_Grade']}<br>"
            f"Total Hispanic: {row['Total_HISPANIC']}<br>"
            f"Ratio: {row['ratio']}"
        ),
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(marker_cluster)


m.save( 'texas_census_tract_hispanic_map.html')




'texas_census_tract_hispanic_map.html'

In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from math import ceil



final_df['ratio'] = final_df['ratio'].apply(lambda x: ceil(x + 1))

grouped_df = final_df.groupby('Census_Tract').agg({
    'City': 'first',
    'Latitude': 'first',
    'Longitude': 'first',
    'Hispanic_Enrolled_12th_Grade': 'sum',
    'Total_HISPANIC': 'sum',
    'ratio': 'first'
}).reset_index()

m = folium.Map(location=[31.9686, -99.9018], zoom_start=6)

marker_cluster = MarkerCluster().add_to(m)

for index, row in grouped_df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=(
            f"City: {row['City']}<br>"
            f"Census Tract: {row['Census_Tract']}<br>"
            f"Hispanic Enrolled 12th Grade: {row['Hispanic_Enrolled_12th_Grade']}<br>"
            f"Total Hispanic: {row['Total_HISPANIC']}<br>"
            f"Ratio: {row['ratio']}"
        ),
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(marker_cluster)

m.save('texas_census_tract_hispanic_map5.html')



'texas_census_tract_hispanic_map5.html'

In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import matplotlib.cm as cm
import matplotlib.colors as colors
from math import ceil


final_df['ratio'] = final_df['ratio'].apply(lambda x: ceil(x + 1))

grouped_df = final_df.groupby('Census_Tract').agg({
    'City': 'first',
    'Latitude': 'first',
    'Longitude': 'first',
    'Hispanic_Enrolled_12th_Grade': 'sum',
    'Total_HISPANIC': 'sum',
    'ratio': 'first'
}).reset_index()

min_ratio = grouped_df['ratio'].min()
max_ratio = grouped_df['ratio'].max()
norm = colors.Normalize(vmin=min_ratio, vmax=max_ratio)
colormap = cm.ScalarMappable(norm=norm, cmap='viridis')

def get_marker_color(ratio):
    color = colormap.to_rgba(ratio)
    return colors.rgb2hex(color)

m = folium.Map(location=[31.9686, -99.9018], zoom_start=6)

marker_cluster = MarkerCluster().add_to(m)

for index, row in grouped_df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=(
            f"City: {row['City']}<br>"
            f"Census Tract: {row['Census_Tract']}<br>"
            f"Hispanic Enrolled 12th Grade: {row['Hispanic_Enrolled_12th_Grade']}<br>"
            f"Total Hispanic: {row['Total_HISPANIC']}<br>"
            f"Ratio: {row['ratio']}"
        ),
        icon=folium.Icon(color=get_marker_color(row['ratio']), icon='info-sign')
    ).add_to(marker_cluster)
#didnt work but also where i left off
legend_html = '''
<div style="position: fixed;
            bottom: 50px; left: 50px; width: 150px; height: 90px;
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color:white; opacity:0.8;">
    <i class="fa fa-square fa-2x" style="color: {low_color}"></i>&nbsp; Min {min_ratio} <br>
    <i class="fa fa-square fa-2x" style="color: {high_color}"></i>&nbsp; Max {max_ratio} <br>
</div>
'''.format(low_color=colors.rgb2hex(colormap.to_rgba(min_ratio)),
           high_color=colors.rgb2hex(colormap.to_rgba(max_ratio)),
           min_ratio=min_ratio,
           max_ratio=max_ratio)

m.get_root().html.add_child(folium.Element(legend_html))


m.save('Texas_census_tract_hispanic_map_continuous_legend.html')



<ipython-input-20-dd095460b4ca>:50: UserWarning: color argument of Icon should be one of: {'darkblue', 'darkred', 'white', 'purple', 'beige', 'lightgray', 'darkpurple', 'blue', 'cadetblue', 'lightred', 'lightgreen', 'darkgreen', 'gray', 'orange', 'pink', 'green', 'lightblue', 'red', 'black'}.
  icon=folium.Icon(color=get_marker_color(row['ratio']), icon='info-sign')


'exas_census_tract_hispanic_map_continuous_legend.html'